# Lesson 5: Human in the Loop

In [ ]:
from utils.helpers import set_api_keys_env
set_api_keys_env()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from uuid import uuid4

In [ ]:


"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [ ]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
tool = TavilySearchResults(max_results=3)

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
thread1 = {"configurable": {"thread_id": "1"}}
thread2 = {"configurable": {"thread_id": "2"}}

In [ ]:
with SqliteSaver.from_conn_string(":memory:") as mem:
    abot = Agent(model, [tool], system=prompt, checkpointer=mem)
    messages = [HumanMessage(content="Whats the weather in Banglore?")]
    for event in abot.graph.stream({'messages': messages}, thread1):
        for v in event.values():
            print(v)
            print("-------------------------------------------------------------")
    print("Info Current State -------------------------------------------------------------")
    curr_state = abot.graph.get_state(thread1)
    print("Current state:", curr_state)
    print("Next State: ", curr_state.next)
    print("Continue after Interrupt -------------------------------------------------------------")
    for event in abot.graph.stream(None, thread1):
        for v in event.values():
            print(v)
            print("-------------------------------------------------------------")
    curr_state = abot.graph.get_state(thread1)
    print("Current state:", curr_state)
    print("Next State: ", curr_state.next)

    

In [ ]:
with SqliteSaver.from_conn_string(":memory:") as mem:
    abot = Agent(model, [tool], system=prompt, checkpointer=mem)
    messages = [HumanMessage(content="Whats the weather in Banglore?")]
    for event in abot.graph.stream({"messages": messages}, thread2):
        for v in event.values():
            print(v)
            print("-------------------------------------------------------------")
    print("Info Current State -------------------------------------------------------------")
    curr_state = abot.graph.get_state(thread2)
    print("Current state:", curr_state)
    print("Next State: ", curr_state.next)
    print("Continue after Interrupt -------------------------------------------------------------")
    while abot.graph.get_state(thread2).next:
        print("Current State: \n", abot.graph.get_state(thread2),"\n")
        _input = input("proceed?")
        if _input != "y":
            print("aborting")
            break
        for event in abot.graph.stream(None, thread2):
            for v in event.values():
                print(v)
                print("-------------------------------------------------------------")

### Updating State

In [ ]:
# mem1 = SqliteSaver.from_conn_string(":memory:")
messages = [HumanMessage(content="Whats the weather in Chembakur, Andhra Pradesh?")]
thread3 = {"configurable": {"thread_id": "3"}}
with SqliteSaver.from_conn_string(":memory:") as mem1:
    abot = Agent(model, [tool], system=prompt, checkpointer=mem1)
    for event in abot.graph.stream({"messages": messages}, thread3):
        for v in event.values():
            print(v)
            print("-------------------------------------------------------------")

    print("Info Current State -------------------------------------------------------------")
    curr_state = abot.graph.get_state(thread3)
    print("Current state:", curr_state)

    last_message = curr_state.values['messages'][-1]
    print("Last message:", last_message)
    print("Tool Calls:", last_message.tool_calls)

    tool_ = last_message.tool_calls[0]
    tool_id = tool_['id']
    curr_state.values['messages'][-1].tool_calls = [{
        'name': tool.name,
        'args': {'query': 'weather in MadanaPalle, Andhra Pradesh'},
        'id': tool_id,
    }]
    print(f"before update: {abot.graph.get_state(thread3)}")

    abot.graph.update_state(thread3, curr_state.values)

    print(f"after update: {abot.graph.get_state(thread3)}")
    ### Run the graph after update
    print(f"CUrrent State after update")
    curr_state = abot.graph.get_state(thread3)
    print("Current state:", curr_state)
    print("Next State: ", curr_state.next)
    print("Continue after Interrupt -------------------------------------------------------------")

    for event in abot.graph.stream(None, thread3):
        for v in event.values():
            print(v)
            print("-------------------------------------------------------------")

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
messages = [HumanMessage(content="Whats the weather in Chembakur, Andhra Pradesh?")]
thread3 = {"configurable": {"thread_id": "3"}}
inmem = InMemorySaver()
abot = Agent(model, [tool], system=prompt, checkpointer=inmem)
for event in abot.graph.stream({"messages": messages}, thread3):
    for v in event.values():
        print(v)
        print("-------------------------------------------------------------")
print("===============================================================================================================")
print("======================================================== Before Update ========================================")
print("===============================================================================================================")
for state in abot.graph.get_state_history(thread3):
    print("State: ", state)
    print("messages: ", len(state.values['messages']))
    print("-------------------------------")
    # break

curr_state = abot.graph.get_state(thread3)
last_message = curr_state.values['messages'][-1]
tool_ = last_message.tool_calls[0]
tool_id = tool_['id']
curr_state.values['messages'][-1].tool_calls = [{
    'name': tool.name,
    'args': {'query': 'weather in MadanaPalle, Andhra Pradesh'},
    'id': tool_id,
}]
abot.graph.update_state(thread3, curr_state.values)

print("===============================================================================================================")
print("======================================================== After Update ========================================")
print("===============================================================================================================")
states = []
for state in abot.graph.get_state_history(thread3):
    states.append(state)
    print("**State: ", state)
    print("**messages: ", len(state.values['messages']))
    print("-------------------------------")
    # break

### Time Travel 

In [ ]:
to_replay1 = states[0]
to_replay2 = states[1]

In [ ]:
to_replay1.values['messages'][-1].tool_calls

In [ ]:
to_replay2.values['messages'][-1].tool_calls

In [ ]:
for event in abot.graph.stream(None, to_replay1.config):
    for k, v in event.items():
        print(f" Event {k}: {v}")
        print("-------------------------------------------------------------")

In [ ]:
len(list(abot.graph.get_state_history(thread3)))


In [ ]:
curr_state = abot.graph.get_state(thread3)
curr_state.values['messages']

In [ ]:
for event in abot.graph.stream(None, to_replay2.config):
    for k, v in event.items():
        print(f" Event {k}: {v}")
        print("-------------------------------------------------------------")

In [ ]:
len(list(abot.graph.get_state_history(thread3)))

In [ ]:
curr_state = abot.graph.get_state(thread3)
curr_state.values['messages']

In [ ]:
_id = to_replay1.values['messages'][-1].tool_calls[0]['id']
to_replay1.values['messages'][-1].tool_calls = [{
    'name': tool.name,
    'args': {'query': 'weather in Chintamani, Karnataka'},
    'id': _id,
}]

In [ ]:
branch_state = abot.graph.update_state(to_replay1.config, to_replay1.values)
# branch_state

In [ ]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        print(f" Event {k}: {v}")
        print("-------------------------------------------------------------")

In [ ]:
len(list(abot.graph.get_state_history(thread3)))

In [ ]:
curr_state = abot.graph.get_state(thread3)
curr_state.values['messages']

In [ ]:
_id = to_replay1.values['messages'][-1].tool_calls[0]['id']
state_update = {'messages': [ToolMessage(tool_call_id=_id, name=tool.name, content="Weather in Chembakur, Andhra Pradesh is 30C")]}

In [ ]:
branch_and_add = abot.graph.update_state(
    to_replay1.config,
    state_update,
    as_node="action"
)

In [ ]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(f" Event {k}: {v}")
        print("-------------------------------------------------------------")

In [ ]:
len(list(abot.graph.get_state_history(thread3)))

In [ ]:
curr_state = abot.graph.get_state(thread3)
curr_state.values['messages']